### Algoritmo Random Forest Classifier

Algumas características sobre o nome das features:
- O nome dos atributos indica o grupo ao qual pertence (ind, reg, car);
- Os prefixos bin e cat indicam atributos binários e categóricos, respectivamente;
- Atributos sem os prefixos citados podem ser ordinais ou contínuos;
- Atributos com -1 indicam dado faltante (missing); e
- A coluna 'target' indica se houve sinistro para apólice ou não.

Iniciaremos com o Pré Processamento conforme visto em sala

In [1]:
# importa os arquivos de treino e teste
import pandas as pd

treino = pd.read_csv('train.csv')
teste = pd.read_csv('test.csv')
treino.describe()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,5.952120e+05,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,...,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000
mean,7.438036e+05,0.036448,1.900378,1.358943,4.423318,0.416794,0.405188,0.393742,0.257033,0.163921,...,5.441382,1.441918,2.872288,7.539026,0.122427,0.627840,0.554182,0.287182,0.349024,0.153318
std,4.293678e+05,0.187401,1.983789,0.664594,2.699902,0.493311,1.350642,0.488579,0.436998,0.370205,...,2.332871,1.202963,1.694887,2.746652,0.327779,0.483381,0.497056,0.452447,0.476662,0.360295
min,7.000000e+00,0.000000,0.000000,-1.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.719915e+05,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.435475e+05,0.000000,1.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,1.115549e+06,0.000000,3.000000,2.000000,6.000000,1.000000,0.000000,1.000000,1.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,1.488027e+06,1.000000,7.000000,4.000000,11.000000,1.000000,6.000000,1.000000,1.000000,1.000000,...,19.000000,10.000000,13.000000,23.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


É importante verificar se existem registros duplicados. Registros duplicados em uma mesma classe não são relevantes para a maioria dos métodos, e podem economizar processamento quando removidos. **Já registros duplicados com classes diferentes podem confundir praticamente todos os métodos, é extremamente importante removê-los.**

In [2]:
print('Antes:', treino.shape)
treino.drop_duplicates()
print('Depois:', treino.shape)

Antes: (595212, 59)
Depois: (595212, 59)


Vamos criar metadados para os conjuntos para melhor organizá-los e definir o propósito de cada atribudo e onde pode ser usado.

In [3]:
data = []
for f in treino.columns:
    # definindo o uso (entre rótulo, id e atributos)
    if f == 'target':
        role = 'target' # rótulo
    elif f == 'id':
        role = 'id'
    else:
        role = 'input' # atributos
         
    # definindo o tipo do dado
    if 'bin' in f or f == 'target':
        level = 'binary'
    elif 'cat' in f or f == 'id':
        level = 'nominal'
    elif treino[f].dtype == float:
        level = 'interval'
    elif treino[f].dtype == int:
        level = 'ordinal'
        
    # mantem keep como verdadeiro pra tudo, exceto id
    keep = True
    if f == 'id':
        keep = False
    
    # cria o tipo de dado
    dtype = treino[f].dtype
    
    # cria dicionário de metadados
    f_dict = {
        'varname': f,
        'role': role,
        'level': level,
        'keep': keep,
        'dtype': dtype
    }
    data.append(f_dict)
    
meta = pd.DataFrame(data, columns=['varname', 'role', 'level', 'keep', 'dtype'])
meta.set_index('varname', inplace=True)

Para visualizar o atributo e todos seus metadados, basta mostrar a variável meta:

In [4]:
meta

,role,level,keep,dtype
varname,,,,
id,id,nominal,False,int64
target,target,binary,True,int64
ps_ind_01,input,binary,True,int64
ps_ind_02_cat,input,nominal,True,int64
ps_ind_03,input,nominal,True,int64
ps_ind_04_cat,input,nominal,True,int64
ps_ind_05_cat,input,nominal,True,int64
ps_ind_06_bin,input,binary,True,int64
ps_ind_07_bin,input,binary,True,int64


Com essa estrutura de metadados, fica fácil consultar quais colunas quer se manter e que são nominais, por exemplo:

In [4]:
meta[(meta.level == 'nominal') & (meta.keep)].index

Index(['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_car_01_cat',
       'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat',
       'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat',
       'ps_car_10_cat', 'ps_car_11_cat'],
      dtype='object', name='varname')

Da mesma forma, seria possível contar os atributos por tipo de uso e dado:

In [5]:
pd.DataFrame({'count' : meta.groupby(['role', 'level'])['role'].size()}).reset_index()

,role,level,count
0,id,nominal,1
1,input,binary,17
2,input,interval,10
3,input,nominal,14
4,input,ordinal,16
5,target,binary,1


## Valores faltantes

Conforme já mencionado, os valores faltantes são indicados por -1, então é importante saber quais colunas têm valores faltantes e em qual proporção.

In [6]:
atributos_missing = []

for f in treino.columns:
    missings = treino[treino[f] == -1][f].count()
    if missings > 0:
        atributos_missing.append(f)
        missings_perc = missings/treino.shape[0]
        
        print('Atributo {} tem {} amostras ({:.2%}) com valores faltantes'.format(f, missings, missings_perc))
        
print('No total, há {} atributos com valores faltantes'.format(len(atributos_missing)))

Atributo ps_ind_02_cat tem 216 amostras (0.04%) com valores faltantes
Atributo ps_ind_04_cat tem 83 amostras (0.01%) com valores faltantes
Atributo ps_ind_05_cat tem 5809 amostras (0.98%) com valores faltantes
Atributo ps_reg_03 tem 107772 amostras (18.11%) com valores faltantes
Atributo ps_car_01_cat tem 107 amostras (0.02%) com valores faltantes
Atributo ps_car_02_cat tem 5 amostras (0.00%) com valores faltantes
Atributo ps_car_03_cat tem 411231 amostras (69.09%) com valores faltantes
Atributo ps_car_05_cat tem 266551 amostras (44.78%) com valores faltantes
Atributo ps_car_07_cat tem 11489 amostras (1.93%) com valores faltantes
Atributo ps_car_09_cat tem 569 amostras (0.10%) com valores faltantes
Atributo ps_car_11 tem 5 amostras (0.00%) com valores faltantes
Atributo ps_car_12 tem 1 amostras (0.00%) com valores faltantes
Atributo ps_car_14 tem 42620 amostras (7.16%) com valores faltantes
No total, há 13 atributos com valores faltantes


Observamos que existem dois atributos com porcentagem alta de valores faltantes, então optamos por excluir do arquivo.

In [7]:
# removendo ps_car_03_cat e ps_car_05_cat que tem muitos valores faltantes
vars_to_drop = ['ps_car_03_cat', 'ps_car_05_cat']
treino = treino.drop(vars_to_drop, axis=1)
teste = teste.drop(vars_to_drop, axis=1)
meta.loc[(vars_to_drop),'keep'] = False  # atualiza os metadados para ter como referência (processar o test depois)

In [8]:
from sklearn.preprocessing import Imputer

media_imp = Imputer(missing_values=-1, strategy='mean', axis=0)
moda_imp = Imputer(missing_values=-1, strategy='most_frequent', axis=0)
treino['ps_reg_03'] = media_imp.fit_transform(treino[['ps_reg_03']]).ravel()
treino['ps_car_12'] = media_imp.fit_transform(treino[['ps_car_12']]).ravel()
treino['ps_car_14'] = media_imp.fit_transform(treino[['ps_car_14']]).ravel()
treino['ps_car_11'] = moda_imp.fit_transform(treino[['ps_car_11']]).ravel()

teste['ps_reg_03'] = media_imp.fit_transform(teste[['ps_reg_03']]).ravel()
teste['ps_car_12'] = media_imp.fit_transform(teste[['ps_car_12']]).ravel()
teste['ps_car_14'] = media_imp.fit_transform(teste[['ps_car_14']]).ravel()
teste['ps_car_11'] = moda_imp.fit_transform(teste[['ps_car_11']]).ravel()

### Eliminado as dummy variables

Após o tratamento dos dados faltantes temos que verificar se os dados ordinais estão com representação apropriada. Sabemos que cada valor de um atributo deve ser representado por um conjunto de atributos da mesma distância. É importante verificar se esses dados têm grande variedade de valores ou não, para aplicarmos a separação apenas se for viável.


In [9]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index

for f in v:
    dist_values = treino[f].value_counts().shape[0]
    print('Atributo {} tem {} valores distintos'.format(f, dist_values))

Atributo ps_ind_02_cat tem 5 valores distintos
Atributo ps_ind_04_cat tem 3 valores distintos
Atributo ps_ind_05_cat tem 8 valores distintos
Atributo ps_car_01_cat tem 13 valores distintos
Atributo ps_car_02_cat tem 3 valores distintos
Atributo ps_car_04_cat tem 10 valores distintos
Atributo ps_car_06_cat tem 18 valores distintos
Atributo ps_car_07_cat tem 3 valores distintos
Atributo ps_car_08_cat tem 2 valores distintos
Atributo ps_car_09_cat tem 6 valores distintos
Atributo ps_car_10_cat tem 3 valores distintos
Atributo ps_car_11_cat tem 104 valores distintos


Vamos optar por manter todos atributos e, portanto, gerar o conjunto de atributos que os mantêm à mesma distância:

In [10]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index
print('Antes do one-hot encoding tinha-se {} atributos'.format(treino.shape[1]))
treino = pd.get_dummies(treino, columns=v, drop_first=True)
print('Depois do one-hot encoding tem-se {} atributos'.format(treino.shape[1]))

teste = pd.get_dummies(teste, columns=v, drop_first=True)
missing_cols = set( treino.columns ) - set( teste.columns )
for c in missing_cols:
    teste[c] = 0
    
treino, teste = treino.align(teste, axis=1)

Antes do one-hot encoding tinha-se 57 atributos
Depois do one-hot encoding tem-se 211 atributos


## Depois de todo pré-processamento...

É hora de verificar se tanto treino como teste têm o mesmo tamanho/formato, e aplicar um modelo de classificação já que esse é um problema desse tipo. Vale lembrar que o tamanho do treino e teste pode variar quando você estiver participando de outras competições ou explorando outros conjuntos de dados.

Isso porque na maioria das competições não se tem o *target* do test. Estima-se uma resposta e submete ao Kaggle, por exemplo, para que ele verifique qual foi o resultado final. Então esse tamanho pode variar em 1 entre treino e teste. No nosso caso, como todos os dados vêm de uma mesma fonte para experimentos, é esperado que tenham a mesma quantidade de atributos ou colunas.

In [11]:
print(treino.shape)
print(teste.shape)

(595212, 211)
(892816, 211)


Separamos 50% para os testes

In [12]:
X_train = treino.drop(['id', 'target'], axis=1)
y_train = treino['target']

X_test  = teste.drop(['id', 'target'], axis=1)
y_test  = teste['target']

from sklearn.model_selection import train_test_split
X_train_main, X_train_validate, y_train_main, y_train_validate = train_test_split(X_train,y_train,test_size=0.5,stratify=y_train) 

In [13]:
# Encontrando recurso de importação usando o ExtraTreeClassifier 
from sklearn.ensemble import ExtraTreesClassifier
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

forest = ExtraTreesClassifier(n_estimators=250,
                              random_state=0)
forest.fit(X_train_main, y_train_main)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
# Print the feature ranking
print("Feature ranking:")

for f in range(X_train_main.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))
    

Feature ranking:
1. feature 208 (0.026538)
2. feature 175 (0.026437)
3. feature 13 (0.025733)
4. feature 10 (0.025669)
5. feature 9 (0.025663)
6. feature 207 (0.025182)
7. feature 0 (0.025136)
8. feature 183 (0.025130)
9. feature 202 (0.025109)
10. feature 2 (0.025075)
11. feature 1 (0.024945)
12. feature 12 (0.024607)
13. feature 6 (0.024372)
14. feature 7 (0.024367)
15. feature 5 (0.023837)
16. feature 8 (0.023548)
17. feature 4 (0.023357)
18. feature 3 (0.023264)
19. feature 11 (0.023208)
20. feature 178 (0.022842)
21. feature 206 (0.022297)
22. feature 177 (0.022041)
23. feature 176 (0.021866)
24. feature 174 (0.015829)
25. feature 16 (0.015221)
26. feature 15 (0.014960)
27. feature 18 (0.014520)
28. feature 17 (0.013784)
29. feature 70 (0.011526)
30. feature 19 (0.010621)
31. feature 14 (0.009249)
32. feature 65 (0.008824)
33. feature 179 (0.008734)
34. feature 184 (0.008354)
35. feature 185 (0.008330)
36. feature 180 (0.007754)
37. feature 23 (0.007517)
38. feature 63 (0.007396)


In [14]:
# Selecionando os primerios 28 feature mais importantes
important_feature = []
for f in range(28):
    important_feature.append(indices[f])

print(important_feature)    

[208, 175, 13, 10, 9, 207, 0, 183, 202, 2, 1, 12, 6, 7, 5, 8, 4, 3, 11, 178, 206, 177, 176, 174, 16, 15, 18, 17]


In [15]:
# Final dataframe with only important features
train_copy = treino.drop(['target'],axis=1)
final_train = train_copy.iloc[:,important_feature]
X_train = final_train.values
y_train = treino['target'].values

X_train_main, X_train_validate, y_train_main, y_train_validate = train_test_split(X_train,y_train,test_size=0.2,stratify=y_train) 

Agora iremos utilizar o método Random Forest Classifier nos dados que foram tratados e seperados

In [16]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train_main, y_train_main)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [17]:
clf.score(X_train_validate, y_train_validate)

0.9635509857782482

In [18]:
# Prepare submission file
test_copy = teste.iloc[:,important_feature]
X_test = test_copy.values
predicted_test = clf.score(X_test, y_test)

In [19]:
output = pd.DataFrame({'id': teste['id'].values, 'target': predicted_test})

In [20]:
output.to_csv("submission_outputMARI.csv", index=False) 